In [ ]:
import requests
import numpy as np
!pip install "tensorflow>=1.14.0,<2.0"
!pip install "tensorflow-gpu>=1.14.0,<2.0"
!pip install  ampligraph

In [4]:
import ampligraph
import numpy as np
ampligraph.__version__
import pandas as pd
import csv
from ampligraph.datasets import load_from_ntriples
from ampligraph.datasets import load_from_csv

In [5]:
Y=load_from_csv('/content','1000data.csv',sep=',',add_reciprocal_rels=False)

In [4]:
print(Y)

[['Common symptoms' 'include' 'fever']
 ['Management' 'involves' 'treatment']
 ['The World Health Organization' 'declared' 'outbreak']
 ...
 ['black people' 'resisted' 'slavery']
 ['Leading Black activists' 'included' 'Olaudah Equiano']
 ['Supporters' 'included' 'workers']]


In [6]:
Y.shape

(978, 3)

In [6]:
entities = np.unique(np.concatenate([Y[:, 0], Y[:, 2]]))
entities.size

1626

In [7]:
relations = np.unique(Y[:, 1])
relations.size

448

In [7]:
from ampligraph.evaluation import train_test_split_no_unseen 

Y_train, Y_test = train_test_split_no_unseen(Y, test_size=25) 

In [9]:
from ampligraph.evaluation import create_mappings

In [11]:
from ampligraph.evaluation import to_idx

In [12]:
Y_train.shape

(953, 3)

In [8]:
pd.DataFrame(Y_train).to_csv("1000train.csv")


In [ ]:
pd.DataFrame(Y_test).to_csv("1000test.csv")

In [27]:
X=create_mappings(Y_train[])

In [28]:
print(X)

({'include': 0}, {'Common symptoms': 0, 'fever': 1})


In [21]:
from ampligraph.latent_features import TransE

In [35]:
model = TransE(batches_count=10, 
                seed=0, 
                epochs=20, 
                k=10, 
                eta=10,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [36]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
model.fit(Y_train, early_stopping = False)

Average Loss:   1.910518: 100%|██████████| 20/20 [00:00<00:00, 30.30epoch/s]


Getting the true triples Score for one  positive triple


In [22]:
import pandas as pd
from sklearn import preprocessing

from statistics import mean
data = pd.read_csv('1000train.csv',header=None)
data.head()
arr =[]

type(data)

for i,j,k in zip(data[0],data[1],data[2]):
  print(i,j,k)
  H1=model.get_embeddings([i],embedding_type='entity')
  T1=model.get_embeddings([k],embedding_type='entity')
  R1=model.get_embeddings([j], embedding_type='relation')
  df3=np.add(H1,R1)-T1
  normalized = preprocessing.normalize(df3)
  # print("Normalized Data = ", normalized)
  ar=abs(normalized)
  # print(ar)
  # break
  for i in ar:
    Score=mean(i)
    # print(Score)
    arr.append(Score)

print(arr)




Common symptoms include fever
Management involves treatment
The World Health Organization declared outbreak
common symptoms include cough
minority develop noticeable symptoms
Cardiovascular complications include heart failure
Sputum carry large amounts
indicative suggest underlying immunopathology
people have classical serum biomarkers
The US Food and Drug Administration approved test
guidelines recommend medication
Face coverings limit volume
the United States Environmental Protection Agency maintains list
Intensivists compiled treatment recommendations
availability affect mortality
study reported earliest date
Official publications reported earliest onset
Wuhan Central Hospital sent bronchoalveolar lavage fluid BAL sample
the National Health Commission of China issued notice
China reported nearly 140 new cases
Italy overtook China
the United States overtaken China
ratio reflects number
the World Health Organization reported estimates
random antibody testing suggested national IFR
Ear

In [23]:
import pandas
df = pandas.DataFrame(data={"col1": arr})
df.to_csv("./scores.csv", sep=',',index=False)

In [70]:
T1=model.get_embeddings(['fever'],embedding_type='entity')

In [75]:
T1

array([[-0.08529831, -0.09325703,  0.06257331,  0.1132529 , -0.07776085,
         0.07245085,  0.11292059,  0.16094959, -0.13164912,  0.09897169]],
      dtype=float32)

In [71]:
R1=model.get_embeddings(['include'], embedding_type='relation')

In [76]:
R1

array([[-0.25193202, -0.25074363,  0.19484803, -0.36466917, -0.34054306,
         0.27808255,  0.2745251 , -0.29404992,  0.26237684,  0.3203629 ]],
      dtype=float32)

In [79]:
df3=np.add(H1,R1)-T1

In [80]:
df3

array([[-0.07887746, -0.07453818,  0.17261004, -0.5732379 , -0.1398688 ,
         0.30134416,  0.3000772 , -0.6219989 ,  0.24947348,  0.31620544]],
      dtype=float32)

In [81]:
from sklearn import preprocessing
normalized = preprocessing.normalize(df3)
print("Normalized Data = ", normalized)

Normalized Data =  [[-0.07454228 -0.07044149  0.16312324 -0.5417322  -0.13218148  0.284782
   0.28358468 -0.58781326  0.23576218  0.2988265 ]]


In [82]:
abs(normalized)

array([[0.07454228, 0.07044149, 0.16312324, 0.5417322 , 0.13218148,
        0.284782  , 0.28358468, 0.58781326, 0.23576218, 0.2988265 ]],
      dtype=float32)

In [83]:
from statistics import mean

In [88]:
ar=abs(normalized)


In [89]:
for i in ar:
  Score=mean(i)

In [90]:
print(Score)

0.26727894


Getting Score for negative triple for 1 triple

In [105]:
HC1=model.get_embeddings(['Management'],embedding_type='entity')

In [113]:
df4=np.add(HC1,R1)-(T1)

In [114]:
from sklearn import preprocessing
normalized1 = preprocessing.normalize(df4)
print("Normalized Data = ", normalized1)


Normalized Data =  [[-0.22417955 -0.26960897  0.1529488  -0.6281778  -0.3603617   0.11867211
   0.1420579  -0.38711116  0.35964853  0.12541045]]


In [115]:
from statistics import mean
ar1=abs(normalized1)
for i in ar1:
  SC1=mean(i)
print(SC1)  

0.2768177


Getting Score for negative triple 2

In [116]:
HC2=model.get_embeddings(['The World Health Organization'],embedding_type='entity')

In [117]:
df5=np.add(HC2,R1)-(T1)

In [118]:
from sklearn import preprocessing
normalized2 = preprocessing.normalize(df5)
print("Normalized Data = ", normalized2)


Normalized Data =  [[-0.29725972 -0.3215953   0.20838955 -0.6261432  -0.08626258  0.2783635
   0.07863998 -0.25399244  0.33948916  0.3191471 ]]


In [119]:
from statistics import mean
ar2=abs(normalized2)
for i in ar2:
  SC2=mean(i)
print(SC2)  

0.28092825


Getting Score for negative triple 3

In [120]:
HC3=model.get_embeddings(['Cardiovascular complications'],embedding_type='entity')

In [121]:
df6=np.add(HC3,R1)-(T1)

In [122]:
from sklearn import preprocessing
normalized3 = preprocessing.normalize(df6)
print("Normalized Data = ", normalized3)


Normalized Data =  [[-0.07828192 -0.23142861  0.20564345 -0.4176538  -0.15901236  0.32526264
   0.27972317 -0.34810492  0.4761594   0.40787038]]


In [124]:
from statistics import mean
ar3=abs(normalized3)
for i in ar3:
  SC3=mean(i)
print(SC3)  

0.29291406


Getting Score for negative triple 4

In [125]:
HC4=model.get_embeddings(['Wuhan Central Hospital'],embedding_type='entity')

In [126]:
df7=np.add(HC4,R1)-(T1)

In [127]:
from sklearn import preprocessing
normalized4 = preprocessing.normalize(df7)
print("Normalized Data = ", normalized4)


Normalized Data =  [[-0.01649378 -0.09566376  0.10037253 -0.62084883 -0.1709174   0.11852055
   0.10909527 -0.42631224  0.58493704  0.12646478]]


In [128]:
from statistics import mean
ar4=abs(normalized4)
for i in ar4:
  SC4=mean(i)
print(SC4)  

0.23696262


In [132]:
import numpy as np
from ampligraph.evaluation.metrics import rank_score
y_pred = np.array([SC1, SC2, Score,SC3,SC4])
y_true = np.array([0, 0, 1, 0,0])
rank_score(y_true, y_pred)

4

In [37]:
from ampligraph.latent_features import save_model, restore_model

In [38]:
save_model(model, './covid_model.pkl')

In [ ]:
restored_model = restore_model('./covid1_model.pkl')

In [ ]:
restored_model2 = restore_model('./covid2_model.pkl')

In [ ]:
restored_model3 = restore_model('./covid3_model.pkl')

In [22]:
print(model.trained_model_params[1].shape)

(448, 10)


In [23]:
Z=pd.DataFrame(model.trained_model_params[1])

In [30]:
 labels = list(model.rel_to_idx.keys())

In [31]:
print(labels)

['" tweets', '#NAME?', '(', 'Asked', 'Detects', 'Growing up', 'JTO', 'Leading AIDS activist', 'On', 'See also.2020', 'Suppose that', 'The term', 'absorb', 'accompanied by', 'accomplished by', 'accused', 'achieve', 'add', 'address', 'adds', 'adhere to', 'adopted', 'advocated', 'affect', 'affected', 'affected with', 'affects', 'affects up to 9%', 'agreed upon', 'allow', 'analyzed', 'announced', 'appear to', 'appointed', 'approved', 'are', 'assembled at', 'assigned', 'associated with', 'attacked', 'attain', 'attended', 'attracted', 'attributed to', 'auctioned', 'averages', 'banned', 'be', 'been', 'been over 50 astroviruses', 'believed that', 'blared', 'boosts', 'brings', 'broke', 'brought', 'burnetti', 'calculate', 'called', 'carries', 'carry', 'catch', 'cause', 'caused', 'causes', 'claim', 'classified', 'classifies', 'cleave', 'codon', 'collected', 'commended', 'compared', 'compiled', 'completed', 'comprise', 'condemned', 'condenses', 'conducted', 'confirmed that', 'connected to', 'conqu

In [ ]:
len(labels)

In [40]:
import pandas
df = pandas.DataFrame(data={ "col1": labels})
df.to_csv("./metadata_relation.csv", sep=',',index=True)

In [24]:
print(Z)

            0         1         2  ...         7         8         9
0    0.131249  0.008022  0.058818  ... -0.098750  0.119777  0.067369
1    0.017078 -0.160840  0.119386  ...  0.124448 -0.225880 -0.204799
2    0.068979 -0.114741  0.178402  ...  0.161747  0.109025  0.093548
3   -0.149831 -0.029352 -0.085655  ...  0.009053 -0.117247 -0.017760
4   -0.093829  0.174851  0.051798  ...  0.040282  0.131216  0.152945
..        ...       ...       ...  ...       ...       ...       ...
443 -0.215869  0.044284 -0.073009  ...  0.111019  0.062650  0.128320
444 -0.015690  0.172577 -0.098273  ... -0.218828 -0.134722  0.178702
445 -0.087101  0.171170  0.099765  ...  0.164045  0.138609  0.089549
446  0.022463 -0.023995  0.169914  ...  0.117994  0.139173  0.119649
447 -0.084837  0.049492 -0.106706  ...  0.042729 -0.049669  0.125297

[448 rows x 10 columns]


In [25]:
Z.to_csv('rela_vectors.csv', index=False)

In [39]:
from ampligraph.evaluation import evaluate_performance

In [ ]:
Z_train=load_from_csv('/content','1300train.csv',sep=',',add_reciprocal_rels=False)


In [ ]:
Z_test=load_from_csv('/content','1000test.csv',sep=',',add_reciprocal_rels=False)

In [40]:
positives_filter =Y

In [41]:
ranks = evaluate_performance(Y_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=False, # corrupt subj and obj separately while evaluating
                             verbose=True)

100%|██████████| 25/25 [00:00<00:00, 269.86it/s]


In [29]:
type(ranks)

numpy.ndarray

In [45]:
df = pd.DataFrame(data=ranks, columns=["Subject(corrupt)", "Object(corrupt)"])

In [46]:
df.to_csv('Test_triple_ranks.csv')

In [47]:
print(ranks)

[[ 711  797]
 [ 331  783]
 [1008  146]
 [ 398   78]
 [  42   38]
 [ 451  562]
 [ 243  244]
 [ 543   58]
 [  81  347]
 [1183  218]
 [ 996  229]
 [ 405 1154]
 [1603 1198]
 [1266 1161]
 [1050 1258]
 [  12   48]
 [ 553  728]
 [ 752  704]
 [ 207  122]
 [ 562  512]
 [ 537  280]
 [  16   33]
 [ 629  337]
 [ 118 1549]
 [ 235  485]]


In [48]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
mr = mr_score(ranks)
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))
hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.01
MR: 540.02
Hits@10: 0.00
Hits@3: 0.00
Hits@1: 0.00


In [ ]:
from ampligraph.utils import create_tensorboard_visualizations

In [ ]:
create_tensorboard_visualizations(model,r"./")

In [98]:
X_unseen = np.array([
    ['Common symptoms', 'include', 'fever'],
  ['black musician', 'wears', 'royal livery'],
   ['Common effects', 'include', 'chest infections'],
 #   ['pandemic', 'include', 'global social economic disruption']

])

In [19]:
from ampligraph.discovery import discover_facts
from ampligraph.discovery import query_topn


In [99]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [20]:
ampligraph.discovery.query_topn(model, top_n=10, head='common symptoms', relation='include', tail=None, ents_to_consider=None, rels_to_consider=None)

(array([['common symptoms', 'include', 'Asthma'],
        ['common symptoms', 'include', 'black population'],
        ['common symptoms', 'include', 'muscle fatigue'],
        ['common symptoms', 'include', 'role'],
        ['common symptoms', 'include', 'cough'],
        ['common symptoms', 'include', 'Raul Cano'],
        ['common symptoms', 'include', 'protests'],
        ['common symptoms', 'include', 'preplanned analysis'],
        ['common symptoms', 'include',
         'positive stranded RNA virus replication model'],
        ['common symptoms', 'include', 'humans']], dtype='<U70'),
 array([0.03340854, 0.02919142, 0.02852985, 0.02669189, 0.02639998,
        0.02592132, 0.02505897, 0.02500333, 0.02357225, 0.02355267],
       dtype=float32))

In [101]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

100%|██████████| 3/3 [00:00<00:00, 51.76it/s]


In [102]:
scores = model.predict(X_unseen)

In [103]:
from scipy.special import expit
probs = expit(scores)

In [104]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

,statement,rank,score,prob
2,Common effects include chest infections,795,0.006087,0.501522
1,black musician wears royal livery,6,0.010364,0.502591
0,Common symptoms include fever,1,0.033633,0.508407


In [ ]:
def evaluate_performance(X, model, filter_triples=None, verbose=False, filter_unseen=True, entities_subset=None,
                         corrupt_side='s,o', use_default_protocol=False):
    """Evaluate the performance of an embedding model.
    The evaluation protocol follows the procedure defined in :cite:`bordes2013translating` and can be summarised as:
    #. Artificially generate negative triples by corrupting first the subject and then the object.
    #. Remove the positive triples from the set returned by (1) -- positive triples \
    are usually the concatenation of training, validation and test sets.
    #. Rank each test triple against all remaining triples returned by (2).
    With the ranks of both object and subject corruptions, one may compute metrics such as the MRR by
    calculating them separately and then averaging them out.
    Note that the metrics implemented in AmpliGraph's ``evaluate.metrics`` module will already work that way
    when provided with the input returned by ``evaluate_performance``.
    The artificially generated negatives are compliant with the local closed world assumption (LCWA),
    as described in :cite:`nickel2016review`. In practice, that means only one side of the triple is corrupted at a time
    (i.e. either the subject or the object).
    .. note::
        The evaluation protocol assigns the worst rank
        to a positive test triple in case of a tie with negatives. This is the agreed upon behaviour in literature.
    .. hint::
        When ``entities_subset=None``, the method will use all distinct entities in the knowledge graph ``X``
        to generate negatives to rank against. This might slow down the eval. Some of the corruptions may not even
        make sense for the task that one may be interested in.
        For eg, consider the case <Actor, acted_in, ?>, where we are mainly interested in such movies that an actor
        has acted in. A sensible way to evaluate this would be to rank against all the movie entities and compute
        the desired metrics. In such cases, where focus us on particular task, it is recommended to pass the desired
        entities to use to generate corruptions to ``entities_subset``. Besides, trying to rank a positive against an
        extremely large number of negatives may be overkilling.
        As a reference, the popular FB15k-237 dataset has ~15k distinct entities. The evaluation protocol ranks each
        positives against 15k corruptions per side.
    Parameters
    ----------
    X : ndarray, shape [n, 3]
        An array of test triples.
    model : EmbeddingModel
        A knowledge graph embedding model
    filter_triples : ndarray of shape [n, 3] or None
        The triples used to filter negatives.
        .. note::
            When *filtered* mode is enabled (i.e. `filtered_triples` is not ``None``),
            to speed up the procedure, we use a database based filtering. This strategy is as described below:
            * Store the filter_triples in the DB
            * For each test triple, we generate corruptions for evaluation and score them.
            * The corruptions may contain some False Negatives. We find such statements by quering the database.
            * From the computed scores we retrieve the scores of the False Negatives.
            * We compute the rank of the test triple by comparing against ALL the corruptions.
            * We then compute the number of False negatives that are ranked higher than the test triple; and then
              subtract this value from the above computed rank to yield the final filtered rank.
              
            **Execution Time:** This method takes ~4 minutes on FB15K using ComplEx
            (Intel Xeon Gold 6142, 64 GB Ubuntu 16.04 box, Tesla V100 16GB)
    verbose : bool
        Verbose mode
    filter_unseen : bool
        This can be set to False to skip filtering of unseen entities if train_test_split_unseen() was used to
        split the original dataset.
    entities_subset: array-like
        List of entities to use for corruptions. If None, will generate corruptions
        using all distinct entities. Default is None.
    corrupt_side: string
        Specifies which side of the triple to corrupt:
        - 's': corrupt only subject.
        - 'o': corrupt only object.
        - 's+o': corrupt both subject and object.
        - 's,o': corrupt subject and object sides independently and return 2 ranks. This corresponds to the \
        evaluation protocol used in literature, where head and tail corruptions are evaluated separately.
        .. note::
            When ``corrupt_side='s,o'`` the function will return 2*n ranks as a [n, 2] array.
            The first column of the array represents the subject corruptions.
            The second column of the array represents the object corruptions.
            Otherwise, the function returns n ranks as [n] array.
    use_default_protocol: bool
        Flag to indicate whether to use the standard protocol used in literature defined in
        :cite:`bordes2013translating` (default: False).
        If set to `True`, ``corrupt_side`` will be set to `'s,o'`.
        This corresponds to the evaluation protocol used in literature, where head and tail corruptions
        are evaluated separately, i.e. in corrupt_side='s,o' mode
    Returns
    -------
    ranks : ndarray, shape [n] or [n,2] depending on the value of corrupt_side.
        An array of ranks of test triples.
        When ``corrupt_side='s,o'`` the function returns [n,2]. The first column represents the rank against
        subject corruptions and the second column represents the rank against object corruptions.
        In other cases, it returns [n] i.e. rank against the specified corruptions.
    Examples
    --------
    >>> import numpy as np
    >>> from ampligraph.datasets import load_wn18
    >>> from ampligraph.latent_features import ComplEx
    >>> from ampligraph.evaluation import evaluate_performance, mrr_score, hits_at_n_score
    >>>
    >>> X = load_wn18()
    >>> model = ComplEx(batches_count=10, seed=0, epochs=10, k=150, eta=1,
    >>>                 loss='nll', optimizer='adam')
    >>> model.fit(np.concatenate((X['train'], X['valid'])))
    >>>
    >>> filter_triples = np.concatenate((X['train'], X['valid'], X['test']))
    >>> ranks = evaluate_performance(X['test'][:5], model=model,
    >>>                              filter_triples=filter_triples,
    >>>                              corrupt_side='s+o',
    >>>                              use_default_protocol=False)
    >>> ranks
    array([  1, 582, 543,   6,  31])
    >>> mrr_score(ranks)
    0.24049691297347323
    >>> hits_at_n_score(ranks, n=10)
    0.4
